In [7]:
from config import VK_TOKEN

import tqdm
import json
import vk_api
import re
from fuzzywuzzy import process

In [2]:
vk_session = vk_api.VkApi(token=VK_TOKEN, app_id=5820017)
vk_session._auth_token()

vk = vk_session.get_api()

In [3]:
tools = vk_api.VkTools(vk_session)

In [4]:
vegan_recipes = tools.get_all_iter("wall.search", 20, 
                                   values={"owner_id": -35838585, "query": "#vegan", "owners_only": True})

In [6]:
# downloaded_vegan_recipes = []
# for recipe in tqdm.tqdm(vegan_recipes):
#     downloaded_vegan_recipes.append(recipe)
# json.dump(downloaded_vegan_recipes, open("assets/downloaded_vegan_recipes.json", "w"))
downloaded_vegan_recipes = json.load(open("assets/downloaded_vegan_recipes.json"))

In [8]:
def process_recipe(recipe):
    tokens = recipe['text'].split()
    # best occurence of fuzzed ингредиент word
    ingr_word = process.extractOne("ингредиент", tokens)[0]
    ingr_idx = recipe['text'].index(ingr_word)
    # best occurence of fuzzed приготовление word
    prigot_word = process.extractOne("приготовление", tokens)[0]
    prigot_idx = recipe['text'].index(prigot_word)
    
    name = recipe['text'][:ingr_idx].strip()
    name, *description = name.rsplit("\n")
    description = "\n".join(description).strip()
    ingredients = recipe['text'][ingr_idx+len(ingr_word):prigot_idx].split("\n")
    ingredients = [it.strip() for it in ingredients if it.strip()]
    script = recipe['text'][prigot_idx+len(prigot_word):].split("\n")
    script = [it.strip() for it in script if it.strip()]
    return name, description, ingredients, script

In [9]:
for recipe in downloaded_vegan_recipes[5:6]:
    name, description, ingredients, script = process_recipe(recipe)
    print("название", name)
    print("описание", description)
    print("игредиенты", ingredients)
    print("приготовление", script)

название Салат с цветной капустой и горчичной заправкой
описание 
игредиенты ['(на 4-6 порций):', 'Для салата:', '1 кочан цветной капусты, нарезанный на соцветия', '1 ст. нарезанного огурца', '1 ст. нарезанных помидоров', '1\\4 ст. нарезанного зелёного лука', 'Поместите все ингредиенты в миску, перемешайте.', 'Для заправки:', '1\\2 ст. кешью, предварительно замоченных в воде', '1\\2 ст. воды', '1 размоченный финик', '1 ч.л. сухого горчичного порошка', '2 ст.л. кокосового масла', '1-2 ст.л. укропа', 'сок и цедра 1\\2 лимона', '2 измельчённых зубчика чеснока', 'соль и перец по вкусу', '1\\2 ст. нарезанных грецких орехов']
приготовление ['Поместите все ингредиенты, кроме укропа, чеснока, соли и перца в блендер. Взбейте до однородной консистенции. Добавьте оставшиеся цедру, укроп, чеснок, соль и перец. Взбейте ещё несколько раз. Вылейте поверх овощей, присыпьте орешками.', 'Приятного аппетита!', '#салаты@vegarecipes #веган@vegarecipes #vegan@vegarecipes #ПП@vegarecipes']


In [10]:
different_measures = {
    "щеп(отк(а|у|и))?": 2,
    "ч\.\s?л": 5,
    "ст\.\s?л": 15,
    "((ч(айн)?|ст)(олов)?\.?(ые|ых|ая|ой)?)?\s?ложе?к(а|и)?": 5,
    "гр?(амм)?": 1, 
    "зуб(к|чик)?\.?(а|ов)?": 5,
    "головк(а|и)": 5,
    "шт(ук(а|и)?)?": None,
    "кусоч\.?(ка|ек)?": None,
    "веточк(а|и)": 2,
    "кружк(а|и)?": 200,
    "чашк(а|и)?": 200,
    "ст(ак)?\.?(ан|ана)?": 200,
    "банк(а|и)?": 300,
    "ломтик(а|и|ов)?": None,
    "стеб(л(я|ей|евой)|ель(ка)?|)": None,
    "струч(ок|к(а|ов))": None,
    "пуч(ок)?": 50,
    "килограмм": 1000,
    "кг": 1000,
    "по( |\-)вкусу": 10,
    "лист(ы|ья|а|ьев|ов|ик(а|ов)?|очк(а|ов|и)?)": 3, 
    "с?м": None,
    "коч(ан)?": 1000,
    "м?л": None
}
digit_pattern = "\d+((,|\.)?\d*)?"
number_pattern = f"(\d*\s*(½|¼|⅔|⅛|⅓)|{digit_pattern}(\/|\\|\\\\|-)?({digit_pattern})*)"  # group1
measures_pattern = f"({'|'.join(different_measures)})"

In [14]:
re_pattern = re.compile(f"({number_pattern}\s*{measures_pattern}+\.?(?!\S)|{measures_pattern}\.?)")

In [15]:
def parse_number(str_num):
    if "-" in str_num:
        str_num = str_num.split("-")[-1]
    str_num = str_num.replace(",", ".").replace(" ", "")
    str_num = str_num.replace("\\", "/").replace("\\\\", "/")
    str_num = str_num.replace("½", "+0.5").replace("¼", "+0.25").replace("⅔", "+0.66").replace("⅛", "+0.13").replace("⅓", "+0.33")
    return eval(str_num)

In [16]:
def parse_measure(str_measure, different_measures):
    group_measures_pattern = "|".join([f"({p})" for p in different_measures])
    for p, v in different_measures.items():
        match = re.search(p, str_measure)
        if match:
            return v

In [21]:
n_recipes = 0
n_success = 0
for recipe in downloaded_vegan_recipes[5:25]:
    try:
        name, description, ingredients, script = process_recipe(recipe)
        processed_ingredients = []
        for ingr in ingredients:
    #         normalized = " ".join([analyzer.parse(w)[0].normal_form for w in ingr.split()])
            ingr = ingr.replace(";", " ").replace(":", " ").replace(")", " ").replace("(", " ").lower().strip()
            if ingr and ingr[-1] == ",":
                ingr = ingr[:-1]
            matches = re.finditer(re_pattern, ingr)
            ingr_cp = ingr
            split_word_indices = []
            prev_index = 0
            for w in ingr.split():
                ind = ingr_cp.index(w, prev_index)
                split_word_indices.append((ind-1, ind+len(w)))
                prev_index = ind+len(w)
            ingr_matches = []
            for m in matches:
                indices = m.span()
                for split_word_ind in split_word_indices:
                    if split_word_ind[1] > indices[1] and split_word_ind[0] < indices[0]:
                        break
                else:
                    ingr_matches.append(indices)
            
            if not len(ingr_matches):
                food_ingr_match = re.search("[а-яА-Я\s]+", ingr)
                number_match = re.search(number_pattern, ingr)
                if food_ingr_match:
                    ingr = food_ingr_match.group()
                ingr = ingr.strip()
                if number_match:
                    number = parse_number(number_match.group())
                    if len(ingr):
                        processed_ingredients.append((ingr, number))
                else:
                    if len(ingr):
                        processed_ingredients.append(ingr)
            else:
                parsed_ingr_matches = {}
                for ind in ingr_matches:
                    ingr_match = ingr[ind[0]:ind[1]]
                    number_ind = next(re.finditer(number_pattern, ingr_match)).span()
                    number = ingr_match[number_ind[0]:number_ind[1]].strip()
                    number = parse_number(number)
                    measure = ingr_match[:number_ind[0]] + ingr_match[number_ind[1]:].strip()
                    measure_grams = parse_measure(measure, different_measures)
                    if measure_grams:
                        total_grams = number * measure_grams
                    else:
                        total_grams = None
                    food_ingr = ingr[:ind[0]] + ingr[ind[1]:]
                    food_ingr_match = re.search("[а-яА-Я\s]+", food_ingr)
                    if food_ingr_match:
                        food_ingr = food_ingr_match.group()
                    food_ingr = food_ingr.strip()
                    if len(food_ingr):
                        parsed_ingr_matches[food_ingr] = total_grams
                processed_ingredients.append(parsed_ingr_matches)
        if name and processed_ingredients:
            print(name, processed_ingredients)
            n_success += 1
        else:
            pass
        n_recipes += 1
    except Exception as e:
        print(e)
#         print(recipe['text'])
print(f"{n_success/n_recipes:.2%} successful matches")








РОЛЛЫ ИЗ СУХОФРУКТОВ ['грецкий орех', 'чернослив', 'финики', 'кокосовая стружка']





Луковые лепешки на минеральной воде (постные) [{'газированная вода': None}, {'мука': 350}, {'сахар': 15}, {'соль': 5}, {'сухие дрожжи': 5}, {'растительное масло': 45}, {'лук': None}, {'кунжут': 30}]
Рис с кабачками и баклажанами в мультиварке [{'риса': 100}, {'воды': None}, ('луковица', 1), {'томатного соуса или пасты': 15}, {'кабачка': 200}, {'баклажана': 200}, 'соль', 'растительное масло']

Картофельный рулет с грибами. [{'картофель': 400}, {'мука': 300}, {'грибы': 300}, {'лук': None}, {'растительное масло': 70}, {'томатная паста': 30}, {'соль': 5}, {'сахар': 5}, {'сухие прованские травы': 5}, {'кипяченая вода': None}, {'укроп': 50}]



100.00% successful matches
